### OGC WFS

OGC Web Feature Service deals for downloading of vector data. Usually OGC GML format is used for data exchange.

#### Searching for the data in CSW server

NOTE: You can skip this step and use the URL directly

First step: let's find some WFS service in our CSW server:

In [10]:
from owslib.fes import PropertyIsLike, BBox, And, PropertyIsEqualTo
from owslib.csw import CatalogueServiceWeb

thecsw = CatalogueServiceWeb('http://geoportal.gov.cz/php/micka/csw/index.php')

wfs_query = PropertyIsLike('csw:AnyText', 'WFS')
geology_query = PropertyIsLike('csw:AnyText', 'Geology')
service_query = PropertyIsLike('apiso:type', 'service')
geology_and_wfs = And([geology_query, wfs_query, service_query])
thecsw.getrecords2([geology_and_wfs], esn='full')

print(thecsw.results)

for recid in thecsw.records:
    record = thecsw.records[recid]
    print(recid, record.title)

{'matches': 1, 'returned': 1, 'nextrecord': 0}
575a4ef6-2f74-43ed-9352-6f400a010852 ČGS Podložní a povrchová geologie - OneGeology-Europe - anglická verze


Let's have a look at WFS data in czech Geology survey

In [11]:
geology = thecsw.records['575a4ef6-2f74-43ed-9352-6f400a010852']
print("Abstract:\n %s\n" % geology.abstract)

print("Identifier: %s" % geology.identifiers[1]['identifier'])

Abstract:
 INSPIRE prohlížecí služba zobrazující datovou sadu GeoCR500, která byla vytvořena pro účely projektu OneGeology-Europe. Aktualizovaná v rámci projektu EGDI v roce 2016.

Identifier: http://inspire.geology.cz/geoserver/wms?service=WMS&version=1.3.0&request=Getcapabilities


#### Download data using Web Feature Service
Let's read some data

In [15]:
from owslib import wfs as webfeatureservice
url = "http://inspire.geology.cz/geoserver/wms?service=WMS&version=1.3.0&request=Getcapabilities"
geology_wfs = webfeatureservice.WebFeatureService(geology.identifiers[1]['identifier'])



/usr/lib/python3/dist-packages/owslib/util.py:541: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if elements:


Service metadta (Capabilities):

In [17]:
capabilities = geology_wfs.getcapabilities()
print("URL: %s" % capabilities.geturl())
print("Name: %s" % geology_wfs.provider.name)
print("Title: %s" % geology_wfs.identification.title)
print("Keywords: %s" % geology_wfs.identification.keywords[0])
print("Fees: %s" % geology_wfs.identification.fees)
print("Abstract: %s" % geology_wfs.identification.abstract)

URL: http://inspire.geology.cz/geoserver/wms?service=WFS&request=GetCapabilities&version=1.0.0
Name: WFS
Title: Mapové služby geoserveru České geologické služby-WFS
Keywords: WFS, WMS, GEOSERVER
Fees: NONE
Abstract: Mapvé služby České geologické služby publikované pomocí geoserveru jsou převážně určeny pro mezinárodní projekty, které vyžadují specifická řešení a nelze při nich použít standardních služeb publikovaných na mapovém serveru České geologické služby (http://mapy.geology.cz)


Print list of available layers 

In [38]:
for i in geology_wfs.contents:
    print("\n#### {} ####".format(i))
    print(geology_wfs.contents[i].abstract)


#### gsmlp:CZE_CGS_500k_Fault ####
MappedFeature (spatial objects) whose specification property is of type Geologic structure

#### gsmlp:CZE_CGS_500k_Geology_AGE ####
MappedFeature (spatial objects) whose specification property is of type GeologicUnit.

#### gsmlp:CZE_CGS_500k_Geology_Lito ####
MappedFeature (spatial objects) whose specification property is of type GeologicUnit.

#### ge:GeologicStructure ####
-- Definition --
  Structure
  
  -- Description --
  Legend ...
  Spatial properties are only available through association with a MappedFeature.

#### ge:GeologicUnit ####
-- Definition --
  A volume of rock with distinct characteristics.
  
  -- Description --
  Includes both formal units (i.e. formally adopted and named in an official lexicon) and informal units (i.e. named but not promoted to the lexicon) and unnamed units (i.e. recognisable and described and delineable in the field but not otherwise formalised).
  Spatial properties are only available through association 

Download data from selected layer

In [33]:
identifier = 'gsmlp:CZE_CGS_500k_Fault'
features = geology_wfs.getfeature([identifier])

with open("../../outputs/geology_faults.gml", "w") as out:
    out.write(features.read())



[Download](../../outputs/geology_faults.gml) and open the data using QGIS.

![](images/geology_faults.png)

---

[<- WMS](03_wms.ipynb) | [Working with projections ->](../05_projections/01_projections.ipynb)